In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime
from dateutil import parser
import time

## Function to extract headlines and date-time from a given page

In [76]:
def extract_data(driver):
    headlines = []
    date_times = []

    # Extract data from the page
    for headline_elem, datetime_elem in zip(driver.find_elements(By.CSS_SELECTOR, 'h3.title'), 
                                           driver.find_elements(By.CSS_SELECTOR, 'time')):
        headlines.append(headline_elem.text.strip())

        # Try to parse the date-time string in two ways
        date_time_str = datetime_elem.get_attribute('datetime')

        if date_time_str is not None:
            try:
                # Attempt to parse as Unix timestamp
                unix_timestamp = int(date_time_str)
                parsed_date_time = datetime.utcfromtimestamp(unix_timestamp)
            except ValueError:
                # If parsing as Unix timestamp fails, try the general parser
                parsed_date_time = parser.parse(date_time_str)

            formatted_date_time = parsed_date_time.strftime('%Y-%m-%d %H:%M:%S')
        else:
            formatted_date_time = None
        
        date_times.append(formatted_date_time)

    return headlines, date_times

## Function to scrape data from all pages

In [77]:
def scrape_all_pages(base_url, num_pages):
    all_headlines = []
    all_date_times = []

    driver = webdriver.Chrome()  # Use the appropriate webdriver for your browser
    wait = WebDriverWait(driver, 10)

    for page_num in range(1, num_pages + 1):
        url = f"{base_url}/{page_num}"
        driver.get(url)
        
        # Wait for the headlines to be present
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'h3.title')))

        headlines, date_times = extract_data(driver)
        all_headlines.extend(headlines)
        all_date_times.extend(date_times)

        # Print intermediate results
        print(f"Page {page_num} - Headlines: {headlines}")

        # Click "Show More" button if available
        show_more_button = driver.find_element(By.CSS_SELECTOR, 'button.js-show-more-button')
        if show_more_button.is_displayed():
            show_more_button.click()

            # Wait for the additional content to load
            wait.until(EC.invisibility_of_element_located((By.CSS_SELECTOR, 'div.loading-spinner')))

        # Add a delay to avoid being blocked by the website
        time.sleep(2)

    driver.quit()
    return all_headlines, all_date_times

## Main function 

In [78]:
def main():
    base_url = "https://in.investing.com/equities/hdfc-bank-ltd-news"
    num_pages = 238

    all_headlines, all_date_times = scrape_all_pages(base_url, num_pages)

    # Create a DataFrame
    df = pd.DataFrame({'Headline': all_headlines, 'DateTime': all_date_times})
    
    # Save the DataFrame to a CSV file
    df.to_csv('hdfc_news_data.csv', index=False)

    # Display the DataFrame
    print(df)

## Read ME

#### Note that this is a dynamic web page. This extraction of headlines and datatime may not be extracted in correspondence to its relavite rows.

##### The web pages behaves in pattern. When user loads the base url "https://in.investing.com/equities/hdfc-bank-ltd-news" then only a portion of the news is displayed. However when user scroll's down to the end of the page then more headlines are loaded dynamically upto three pages for each scroll respectively and the base url changes to "https://in.investing.com/equities/hdfc-bank-ltd-news/3". After that the page appears a Show More button at the end of the news headlines section where user can click the button to show more headlines. This cycles repeats until the end of the headlines. This python script is designed to tackle this cycle. Since the page is designes in such a way the extraction of data leaves approx 5 rows blank space for each iteration of page and the datetime leaves two rows blank (See CSV file in excel).

## Executing main function

In [79]:
if __name__ == "__main__":
    main()

Page 1 - Headlines: ['PSU Stocks Surge Upto 10% To Dominate Nifty 50 Gainers As Markets Scale New Highs', 'HDFC Bank Share Price Live Updates: HDFC Bank Closes at Rs 1466.35 with 2.19% Weekly Return', 'Interim Budget seen as neutral to mildly positive for equities', 'HDFC Bank Stocks Live Updates: HDFC Bank Closes at Rs 1462.55 with 6-Month Beta of 0.8861', "BofA Securities maintains HDFC Bank at 'buy' with a price target of ₹1.95K", "Investec maintains HDFC Bank at 'hold' with a price target of ₹1.63K", 'HDFC Bank Stocks Live Updates: HDFC Bank Closes at Rs 1444.3 with -2.18% 3-Month Return', 'Financials not a top-down sector for at least a few quarters; buffet of opportunities in power across caps: Digant Haria', 'FPIs see fifth-highest weekly exit since 2008', 'Black Opal group raises $3 million from SBI and HDFC Bank', 'Sanjiv Bhasin picks 4 outperformers to ride the wave; prefers Jio Finance to Bajaj Finance', 'Senior Citizens’ Savings Scheme (SCSS): Can I deduct penal interest wh

Page 4 - Headlines: ['HDFC Bank seeks Singapore bank license to grow overseas', 'Hot Stocks: 3 concerns behind $13 bn rout in HDFC Bank; brokerages downgrade LTIMindtree', 'HDFC Bank Share Price Today Live Updates: HDFC Bank Records 7.14% Decrease in Returns over the Last Month, Closes at Rs 1537.5', 'Big movers on D-Street: What should investors do with HDFC Bank, TV18 Broadcast and SAIL?', 'HDFC Bank ADRs plunge 9% overnight! More selling ahead for private bank shares?', 'Dalal Street benchmarks tank over 2%, most since June 2022', 'HDFC Bank drops 8.4% on margin worries', "Investec maintains HDFC Bank at 'hold' with a price target of ₹1.63K", 'Nosedive correction in banking stocks impacts market sentiment', 'Domestic equities plunge 2 per cent', 'Sensex plunges more than 1,600 points led by HDFC Bank', 'US Treasury yields rise above 4%, global markets react', 'HDFC Bank sees profit rise, plans HDB Financial IPO by 2025', 'Private sector banks lead losses as Sensex plunges 1,200 poin

Page 7 - Headlines: ['Mcap of 6 of top-10 most valued firms decline by Rs 57,408 cr; TCS, HDFC Bank major laggards', 'HDFC Bank Q3 Update: Advances rise 4.9% QoQ; deposit growth lags', 'HDFC Bank market cap hits Rs 13 trn mark', 'F&O stocks: HDFC Bank, Sun Pharma among 5 stocks with short buildup', "HDFC Bank Q3 Update: Lender's gross advances jump 62% YoY to Rs 24.69 lakh crore", 'HDFC Bank shares Q3 business update, says advances jump 62.4%, deposits 27.7%', 'HDFC Bank Share Price Live Updates: HDFC Bank Closes at Rs 1690.85 with a 6-Month Beta of 0.8861', 'HDFC Bank Stocks Live Updates: HDFC Bank Closes at Rs 1672.9 with a Volume of 167,806 Shares', '3-year POTD interest rate hiked; FD rates of SBI vs ICICI Bank vs HDFC Bank vs PNB vs post office time deposit', 'Sensex sheds 536 pts amid weakness in IT shares and HDFC Bank; Nifty below 21,550', "Paytm, SBI, Axis, ICICI, Kotak Bank, HDFC Bank shares: Bernstein sees up to 47% upside. Here's why", 'Chakri Lokapriya on his top largecap,

Page 9 - Headlines: ['HDFC Bank Share Price Live Updates: HDFC Bank Closes at Rs 1656.55 with Impressive 1-Month Return of 10.06%', 'HDFC Bank in talks for infra bond sale worth Rs 10,000 crore', 'HDFC Bank Share Price Live Updates: HDFC Bank Closes at Rs 1634.6 with 6-Month Beta of 0.8861', 'Chola Finance, Voltas, Kotak Bank & HDFC Bank: What Osho Krishan of Angel One says on these 4 stocks', 'HDFC Bank Share Price Today Live Updates: HDFC Bank Closes at Rs 1651.0 with 1.68% Weekly Return', "RBI Approves HDFC Bank's Product on Offline Retail Payments for Adoption", 'HDFC Bank Vs ICICI Bank Vs PNB: Check Latest FD Interest Rate Comparison', 'Top banks latest bulk deposit interest rates: SBI vs HDFC Bank vs ICICI Bank vs PNB vs Axis Bank', 'HDFC Bank Share Price Today Live Updates: HDFC Bank Closes at Rs 1653.2 with 6-Month Beta of 0.8861', 'Sensex hits record high, led by HDFC Bank and LIC growth', 'Mcap of seven of top-10 most valued firms added ₹3.04 lakh cr; HDFC Bank, LIC biggest g

Page 11 - Headlines: ['Nifty 50 index is shining: here are the top constituents in 2023', 'Credit card spending in India jumps 40% in festive season', 'HDFC Bank appoints V. Srinivasa Rangan as executive director', 'Nifty and Sensex dip slightly, Tesla eyes India investment', 'Jefferies India favors Coal India, Honasa in portfolio reshuffle', 'HDFC Bank Share Price Today Live Updates: HDFC Bank Closes at Rs 1521.3 with a Volume of 75,248 Shares', 'HDFC Bank Share Price Today Live Updates: HDFC Bank Closes at Rs 1512.55 with a 3-Month Return of -4.67%', 'From SBI To HDFC Bank, Check Home Loan Interest Rate; Compare & Decide From 4 Big Banks', 'Goldman Sachs raises Nifty target to 21,800 by end of 2024', 'Sensex dips as banking sector struggles, IT firms gain', 'FIIs adjust holdings in Indian equities, private banks in focus', 'Nifty companies report robust earnings, Coal India EPS forecast surges', 'Market valuation of top Indian firms sees mixed results amid global economic shifts', 'M

...

Page 227 - Headlines: ['Indian shares end lower for fourth day, financials hurt most', 'Indian shares edge up as IT gains on weak rupee', 'India shares lower at close of trade; Nifty 50 down 0.64%', 'Indian shares close lower as HDFC Bank drags', 'Indian shares fall; HDFC Bank plunges as bad loans rise', "India's HDFC Bank net profit rises 21% in June quarter", 'India shares lower at close of trade; Nifty 50 down 0.78%', 'BRIEF-HDFC Bank Announces Delisting Of GDRS From Luxembourg Stock Exchange', 'Multiple Economic Headwinds Cloud Profit Outlook for Indian Companies', 'Q&A-Prospect of lower U.S. rates prompts a re-look at holdings -Denker Capital', 'India shares higher at close of trade; Nifty 50 up 1.20%', 'UPDATE 1-India May inflation hits seven-month high; still room for rate cut', "Indian shares cheer Prime Minister Modi's thumping victory", 'India shares lower at close of trade; Nifty 50 down 0.69%', 'BRIEF-HDFC Bank Approves Sub-Division Of Shares In 1:2 Ratio', "Indian shares r

Page 230 - Headlines: ['India shares lower at close of trade; Nifty 50 down 1.69%', 'India shares lower at close of trade; Nifty 50 down 0.80%', "Indian shares echo weakness in Asia; investors await state polls' verdict", 'Indian shares post highest close in 2 months; HDFC Bank leads', 'Indian shares rise; Infosys top gainer', 'Indian shares flat as metals drag; F&O expiry, econ data eyed', "India's NSE, BSE indexes jump nearly 1 pct", 'Indian shares flat ahead of key macro events', 'Indian shares close at over 6-week high; Reliance Ind, HDFC climb', 'Indian shares hit more than 4-week high on energy, bank boost', 'CORRECTED-Indian shares inch higher as weak oil prices support sentiment', 'Indian shares end higher; private-sector banks gain', 'Indian shares lose steam, firm rupee drags IT', 'Indian shares clock biggest weekly gain since mid-2016; rupee sees best day in 5 yrs', 'Indian shares fall, set for worst month since February 2016', 'India shares higher at close of trade; Nifty 5

Page 233 - Headlines: ['Indian shares rise, snap 2 days of losses', 'Indian shares rebound after 2 days of losses', 'India shares lower at close of trade; Nifty 50 down 0.83%', 'Indian shares steady amid weak global cues', 'BRIEF-Indian Cabinet Approves Additional FDI Of 240 Bln Rupees In HDFC Bank - Minister', "UPDATE 1-India's retail inflation accelerates to four-month high in May", "India's NSE index hits 3-week high; pvt-sector banks rise", 'India shares higher at close of trade; Nifty 50 up 0.86%', 'India shares lower at close of trade; Nifty 50 down 0.63%', 'Indian shares end lower; financials drag', 'Indian shares end lower; HDFC Bank drags', 'Indian shares edge down as banks drag; auto stocks gain', 'RPT-UPDATE 2-Indian economic growth hits highest rate in almost two years', 'UPDATE 2-Indian economic growth hits highest rate in almost two years', 'India shares higher at close of trade; Nifty 50 up 1.15%', 'Indian shares rise over 1 pct to 2-week closing high ahead of GDP data',

Page 236 - Headlines: ['BRIEF-HDFC Bank Dec Qtr Net Profit Up About 20 Pct', 'Indian shares rise; financials gain', 'India shares higher at close of trade; Nifty 50 up 0.26%', 'Indian shares hit record highs as banks rally', 'India shares higher at close of trade; Nifty 50 up 0.82%', 'Indian shares hit record highs; private banks gain', "RPT-India's HDFC to raise $1.75 bln from investors including GIC, KKR", "India's HDFC to raise $1.75 bln from investors including GIC, KKR", 'DIARY-India corporate earnings week ahead', 'India shares lower at close of trade; Nifty 50 down 0.12%', "BRIEF-HDFC Bank Seeks Shareholders' Nod To Raise Funds Up To 240 Bln Rupees", "UPDATE 1-India's HDFC Bank plans to raise up to $3.75 bln from share sale", "India's HDFC Bank plans to raise up to $3.75 bln from share sales", 'BRIEF-HDFC Bank Approves Raising Funds Worth Up To 240 Bln Rupees', 'UPDATE 1-Indian mortgage lender HDFC unveils $2 bln fundraising plan', 'Indian mortgage lender HDFC unveils $2 bln fun

                                                Headline             DateTime
0      PSU Stocks Surge Upto 10% To Dominate Nifty 50...  2024-02-02 00:00:00
1      HDFC Bank Share Price Live Updates: HDFC Bank ...                 None
2      Interim Budget seen as neutral to mildly posit...  2024-02-02 11:46:48
3      HDFC Bank Stocks Live Updates: HDFC Bank Close...                 None
4      BofA Securities maintains HDFC Bank at 'buy' w...  2024-02-01 22:01:39
...                                                  ...                  ...
13055                                                     2017-07-24 08:46:39
13056                                                                    None
13057                                                     2017-07-24 08:20:09
13058                                                                    None
13059                                                     2017-07-24 07:30:00

[13060 rows x 2 columns]
